In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

In [ ]:
path = "chromedriver.exe"
service = webdriver.chrome.service.Service(path)
service.start()
options = Options()
options.add_argument("--headless") 

In [ ]:
tickers = ["MMM","AXP","AAPL","BA","CAT","CVX","CSCO","KO","DIS",
           "XOM","GE","GS","HD","IBM","INTC","JNJ","JPM","MCD","MRK",
           "MSFT","NKE","PFE","PG","TRV","UNH","VZ","V","WMT"]

In [ ]:
def get_financial_statement(ticker,type_of_statement="income_statement",depth=1):
    """
    Parameters
    ----------
    ticker : str
    type_of_statement : str
        DESCRIPTION. either of income_statement, balance_sheet and cashflow_statement. The default is income_statement.
    depth : int
        DESCRIPTION. till what depth of the statement you need to go. if depth is 2, the code will iterate the button finding process twice

    Returns
    -------
    df : dataframe

    """
    if type_of_statement=="income_statement":
        url = "https://finance.yahoo.com/quote/{}/financials?p={}".format(ticker,ticker)
    elif type_of_statement=="balance_sheet":
        url = "https://finance.yahoo.com/quote/{}/balance-sheet?p={}".format(ticker,ticker)
    elif type_of_statement=="cashflow_statement":
        url = "https://finance.yahoo.com/quote/{}/cash-flow?p={}".format(ticker,ticker)

    driver = webdriver.Chrome(service=service, options = options)
    driver.get(url)
    driver.implicitly_wait(0.2)
    
    ### clicking dropdown buttons before scraping   
    clicked_buttons = []
    for i in range(depth):
        buttons = driver.find_elements(By.XPATH,  '//section[@class="main svelte-e2c64s"]//button')
        buttons = [i for i in buttons if i not in clicked_buttons]
        for button in buttons:
            if button.accessible_name in ["","Follow","Quarterly","Annual","prev","next"]:
                pass
            else:
                #WebDriverWait(driver, 0.2).until(EC.element_to_be_clickable(button)).click()
                driver.execute_script("arguments[0].click();", button) #this way of clicking may be required for some of the wrapped buttons
        clicked_buttons+=buttons
    
    table = driver.find_element(By.XPATH,  '//div[@class="tableContainer svelte-1pgoo1f"]')
    text_blob = table.text.split("\n")
    income_st_dir = {}
    last_key = None
    for count, row in enumerate(text_blob):
        if count == 0:
            heading = row.split()
            column_count = len(heading[1:])
        else:
            if count%(column_count+1) == 1:
                income_st_dir[row] = []
                last_key = row
            else:
                income_st_dir[last_key].append(row)        
    df = pd.DataFrame(income_st_dir).T
    df.columns = heading[1:]
    
    for col in df.columns:
        df[col] = df[col].str.replace(r'[,\|s-]', '', regex=True)
        df[col] = pd.to_numeric(df[col], errors='coerce')
    
    driver.close()
    return df

In [ ]:
def get_key_stat(ticker):
    #getting key statistics data from yahoo finance for the given ticker
    url = 'https://finance.yahoo.com/quote/{}/key-statistics'.format(ticker)
    
    driver = webdriver.Chrome(service=service, options = options)
    driver.get(url)
    driver.implicitly_wait(0.2)
    
    table = driver.find_element(By.XPATH,  '//div[@class="table-container svelte-104jbnt"]')
    text_blob = table.text.split("\n")
    financial_metrics = {}
    header = text_blob[0].split()
    
    for item in text_blob[1:]:
        parts = item.split()
        key_end = len(parts) - len(header)
        key = ' '.join(parts[:key_end])
        values = parts[key_end:]
        financial_metrics[key] = dict(zip(header, values))
    
    df = pd.DataFrame(financial_metrics).T
    df = df.replace({'T': 'E+12', 'B': 'E+09', 'M': 'E+06', 'K': 'E+03'}, regex=True)
    df = df.apply(pd.to_numeric, errors='coerce')

    driver.close()
    return df

In [ ]:
#list of tickers whose financial data needs to be extracted
financial_dir = {}
for ticker in tickers:
    try:
        df1 = get_financial_statement(ticker,"income_statement")
        df1 = df1.iloc[:,[0]]
        df1.columns = [ticker]
        df2 = get_financial_statement(ticker,"balance_sheet",3)
        df2 = df2.iloc[:,[0]]
        df2.columns = [ticker]
        df3 = get_financial_statement(ticker,"cashflow_statement",2)
        df3 = df3.iloc[:,[0]]
        df3.columns = [ticker]
        df4 = get_key_stat(ticker)
        df4 = df4.iloc[:,[0]]
        df4.columns = [ticker]
        df = pd.concat([df1,df2,df3,df4])
        financial_dir[ticker] = df
        print("data extracted for ",ticker)
        financial_dir[ticker] = df
    except Exception as e:
        print(ticker,":", e)

In [ ]:
# creating dataframe with relevant financial information for each stock using fundamental data
stats = ["EBITDA",
         "Depreciation Amortization Depletion",
         "Market Cap (intraday)",
         "Net Income",
         "Operating Cash Flow",
         "Capital Expenditure",
         "Current Assets",
         "Current Liabilities",
         "Net PPE",
         "Stockholders' Equity",
         "Long Term Debt And Capital Lease Obligation",
         "Forward Annual Dividend Yield"] # change as required

indx = ["EBITDA","D&A","MarketCap","NetIncome","CashFlowOps","Capex","CurrAsset",
        "CurrLiab","PPE","BookValue","TotDebt","DivYield"]

def info_filter(df,stats,indx):
    """function to filter relevant financial information
       df = dataframe to be filtered
       stats = headings to filter
       indx = rename long headings
       lookback = number of years of data to be retained"""
    for stat in stats:
        if stat not in df.index:
            print("unable to find {} in {}".format(stat,df.columns[0]))
            return
    df_new = df.loc[stats]
    df_new = df_new[~df_new.index.duplicated(keep='first')]
    df_new.rename(dict(zip(stats,indx)),inplace=True)
    return df_new

#applying filtering to the finacials and calculating relevant financial metrics for each stock
transformed_df = {}
for ticker in financial_dir:
    transformed_df[ticker] = info_filter(financial_dir[ticker],stats,indx)
    if transformed_df[ticker] is None:
        del transformed_df[ticker]
        continue
    transformed_df[ticker].loc["EBIT",:] = transformed_df[ticker].loc["EBITDA",:] - transformed_df[ticker].loc["D&A",:]
    transformed_df[ticker].loc["TEV",:] =  transformed_df[ticker].loc["MarketCap",:] + \
                                           transformed_df[ticker].loc["TotDebt",:] - \
                                           (transformed_df[ticker].loc["CurrAsset",:]-transformed_df[ticker].loc["CurrLiab",:])
    transformed_df[ticker].loc["EarningYield",:] =  transformed_df[ticker].loc["EBIT",:]/transformed_df[ticker].loc["TEV",:]
    transformed_df[ticker].loc["FCFYield",:] = (transformed_df[ticker].loc["CashFlowOps",:]-transformed_df[ticker].loc["Capex",:])/transformed_df[ticker].loc["MarketCap",:]
    transformed_df[ticker].loc["ROC",:]  = (transformed_df[ticker].loc["EBITDA",:] - transformed_df[ticker].loc["D&A",:])/(transformed_df[ticker].loc["PPE",:]+transformed_df[ticker].loc["CurrAsset",:]-transformed_df[ticker].loc["CurrLiab",:])
    transformed_df[ticker].loc["BookToMkt",:] = transformed_df[ticker].loc["BookValue",:]/transformed_df[ticker].loc["MarketCap",:]

################################Output Dataframes##############################
final_stats_val_df = pd.DataFrame(columns=transformed_df.keys())
for key in transformed_df:
    final_stats_val_df[key] = transformed_df[key].values.flatten()
final_stats_val_df.set_index(transformed_df[key].index,inplace=True)
    

# finding value stocks based on Magic Formula
final_stats_val_df.loc["CombRank",:] = final_stats_val_df.loc["EarningYield",:].rank(ascending=False,na_option='bottom')+final_stats_val_df.loc["ROC",:].rank(ascending=False,na_option='bottom')
final_stats_val_df.loc["MagicFormulaRank",:] = final_stats_val_df.loc["CombRank",:].rank(method='first')
value_stocks = final_stats_val_df.loc["MagicFormulaRank",:].sort_values()
print("------------------------------------------------")
print("Value stocks based on Greenblatt's Magic Formula")
print(value_stocks)


# finding highest dividend yield stocks
high_dividend_stocks = final_stats_val_df.loc["DivYield",:].sort_values(ascending=False)
print("------------------------------------------------")
print("Highest dividend paying stocks")
print(high_dividend_stocks)

# # Magic Formula & Dividend yield combined
final_stats_val_df.loc["CombinedRank",:] =  final_stats_val_df.loc["EarningYield",:].rank(ascending=False,method='first') \
                                           +final_stats_val_df.loc["ROC",:].rank(ascending=False,method='first')  \
                                           +final_stats_val_df.loc["DivYield",:].rank(ascending=False,method='first')
value_high_div_stocks = final_stats_val_df.T.sort_values("CombinedRank").loc[:,["EarningYield","ROC","DivYield","CombinedRank"]]
print("------------------------------------------------")
print("Magic Formula and Dividend Yield combined")
print(value_high_div_stocks)